In [ ]:
!pip install -qq git+https://github.com/openai/whisper.git 
!pip install git+https://github.com/m-bain/whisperx.git

In [1]:
import whisper
import subprocess #if stripping audio from video file
import datetime #to print time of start and end of analysis

In [ ]:
# Choose and load model size
model_size= "large-v2"
base_model = whisper.load_model(model_size)

In [ ]:
#Path to file that needs to be transcribed
#This should be modified to cycle through files in a folder
path = "STLS_NV.mp3"

In [ ]:
# Strip audio from video file. Not necessary for Whisper, but possible for running other analyses
subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
path= 'audio.wav'

In [ ]:
start = datetime.datetime.now()

whisperx "audio.wav" --model large --language sv --vad_filter --parallel_bs 16 --diarize --align_model "KBLab/wav2vec2-large-voxrex-swedish"

end = datetime.datetime.now()
delta = end - start
print("Runtime was " + str(delta))